In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import dlib

In [2]:
# create a videoCapture object with a video file or a capture device
cap = cv2.VideoCapture("test.mp4")

# initialize dlib's face detector (HOG-based)
detector = dlib.get_frontal_face_detector()

# load model
model = load_model("model.h5")

# check if we will successfully open the file
if not cap.isOpened():
    print("Error opening the file.")
    assert(False)

# read until the end of the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        frame = cv2.resize(frame, (500,400))
        gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
        rects = detector(gray, 0)

        if(len(rects) != 0):
            rect = rects[0]
            top, right, bottom, left = abs(rect.top()),abs(rect.right()),abs(rect.bottom()),abs(rect.left())
            face = frame[top:bottom,left:right]

            face = cv2.resize(face,(300,300))
            cv2.rectangle(frame, (left, top), (right, bottom), (255, 255, 0), 2)
            
            face = np.expand_dims(face,axis=0)
            y = model.predict(face)
        
            if(y[0][0] < 0.5):
                s = "without mask"
            else:
                s = "with mask"
            
            cv2.putText(frame, 
                        s + ", " + str(int((y[0][0])*100)) + "%", 
                        (left-5,top-5), 
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0,0,0), 2)
        
        cv2.imshow("Output", frame)

        if cv2.waitKey(1) == ord('e'):
            break
    else:
        break

# to release software and hardware resources
cap.release()

# to close all windows in imshow ()
cv2.destroyAllWindows()